# Исследование надёжности заёмщиков

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

In [2]:
clients_data = pd.read_csv('/datasets/data.csv')
#убираем заглавные буквы из столбца education
clients_data['education'] = clients_data['education'].str.lower()
print(clients_data.info())
clients_data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В таблице 21 525 строк. Заголовки удобны для работы, типы данных для столбцов определены корректно.
- В столбце days_employed есть отрицательные значения. Такого быть не должно. Кроме того, у пенсионеров явно слишком большой стаж. Предположим, что данные указаны в часах и пересчитаем в дни.
- В столбце education есть как заглавные, так и строчные буквы. Перед анализом нужно будет привести значения к единому виду.
- В столбце total_income значение не округлено до 2 знаков после запятой. Нужно будет обратить внимание, чтобы это не повлияло на расчеты.
- В столбце purpose одинаковые цели записаны по-разному (напр. "покупка жилья", "покупка жилья для семьи" и "операции с жильем"). Нужно привести эти значения к единому виду.

Также в столбцах days_employed и total_income есть пустые значения. Они могли появиться по следующим причинам:

1) Данные не были указаны при заполнении анкеты заемщика

2) У заемщика нет трудового стажа

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
clients_data.loc[clients_data['days_employed'] < 0, 'days_employed'] = -(clients_data.loc[clients_data['days_employed'] < 0, 'days_employed'])
clients_data['days_employed'].min()
clients_data.loc[clients_data['children'] < 0, 'children'] = -(clients_data.loc[clients_data['children'] < 0, 'children'])

#добавляем колонку с годами и проверяем максимальное значение
clients_data['years_employed'] = clients_data['days_employed'] / 365
print(clients_data['years_employed'].max())

#убираем нули в возрасте пенсионеров по медиане для этой группы
rentner_median_age = clients_data.loc[clients_data['income_type'] == 'пенсионер']['dob_years'].median()
print(rentner_median_age)
clients_data.loc[clients_data['income_type'] == 'пенсионер'].fillna(rentner_median_age)

#1083 года - явная ошибка. Отбираем записи, где разница между возрастом и стажем меньше 14
clients_data['difference'] = clients_data['dob_years'] - clients_data['years_employed']
#clients_data[clients_data['difference'] < 14].groupby('income_type')['income_type'].count()

#таких 3535 клиентов, подавляющее большинство - пенсионеры. Явная ошибка в заполнении данных.
#Предположим, что для пенсионеров стаж указан в часах.
clients_data.loc[clients_data['income_type'] == 'пенсионер', 'days_employed'] = clients_data['days_employed'] / 24
clients_data['years_employed'] = clients_data['days_employed'] / 365
clients_data['difference'] = clients_data['dob_years'] - clients_data['years_employed']
#clients_data[clients_data['difference'] < 14].groupby('income_type')['income_type'].count()

#Теперь - самая многочисленная группа - сотрудники (60 записей). Эти строки можно удалить
clients_data.drop(clients_data[clients_data['difference'] < 14].index, inplace=True)

1100.6997273296713
60.0


In [4]:
clients_category = clients_data['income_type'].unique()
#clients_category
for category in clients_category:
    #print(clients_data[clients_data['income_type'] == category])
    median_employment = clients_data.loc[(clients_data['income_type'] == category) & 
                                         (clients_data['days_employed'] > 0)].sort_values(by=[
                                            'days_employed'])['days_employed'].median()
    median_income = clients_data.loc[(clients_data['income_type'] == category) &
                                    (clients_data['total_income'] > 0)].sort_values(by=[
                                        'total_income'])['total_income'].median()
    clients_data.loc[clients_data['income_type'] == category, 'days_employed'] = clients_data.loc[
        clients_data['income_type'] == category, 'days_employed'].fillna(median_employment)
    clients_data.loc[clients_data['income_type'] == category, 'total_income'] = clients_data.loc[
        clients_data['income_type'] == category, 'total_income'].fillna(median_income)

#убираем вспомогательные столбцы
clients_data = clients_data.drop(['years_employed', 'difference'], axis=1)

print(clients_data.info())
clients_data.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20539 entries, 0 to 21524
Data columns (total 12 columns):
children            20539 non-null int64
days_employed       20539 non-null float64
dob_years           20539 non-null int64
education           20539 non-null object
education_id        20539 non-null int64
family_status       20539 non-null object
family_status_id    20539 non-null int64
gender              20539 non-null object
income_type         20539 non-null object
debt                20539 non-null int64
total_income        20539 non-null float64
purpose             20539 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

1. Отрицательные значения в столбце days_employed. Возможно, количество дней считалось как разница между текущей датой и датой начала стажа, и даты начала и конца были перепутаны. В пользу этого говорит количество отрицательных записей: 15906 из 19351. Меняем знак у отрицательных значений.
Кроме того, в столбце days_employed 3529 значений, которые в пересчете на года больше возраста заемщика. Очевидно, что эти значения заполнены некорректно.


2. Пустые значения в столбцах days_employed и total_income. Они могли появиться по следующим причинам:

    1) Данные не были указаны при заполнении анкеты заемщика
    
    2) У заемщика нет трудового стажа
    
Стаж и доход - это числовые значения. Рассчитываем медианы по непустым значениям и заполняем пропуски. Также подставляем медианное значение туда, где стаж больше возраста.

3. Нули в столбце dob_age. Считаем медиану по непустым значениям и заменяем.

### Замена типа данных

In [5]:
clients_data['days_employed'] = clients_data['days_employed'].astype('int')
clients_data['dob_years'] = clients_data['dob_years'].astype('int')
clients_data['total_income'] = clients_data['total_income'].round(decimals=2)
clients_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20539 entries, 0 to 21524
Data columns (total 12 columns):
children            20539 non-null int64
days_employed       20539 non-null int64
dob_years           20539 non-null int64
education           20539 non-null object
education_id        20539 non-null int64
family_status       20539 non-null object
family_status_id    20539 non-null int64
gender              20539 non-null object
income_type         20539 non-null object
debt                20539 non-null int64
total_income        20539 non-null float64
purpose             20539 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

    1) Приводим столбец days_employed к целочисленному формату, т.к. нам неважны доли дня в стаже.
    2) Приводим столбец dob_years к целочисленному (как он был до наших преобразований)
    3) В столбце total_income приводим значения к числовому формату с 2 точками после запятой.

### Обработка дубликатов

In [6]:
clients_data.duplicated().sum()

71

In [7]:
clients_data = clients_data.drop_duplicates(keep='first'
                                           )
clients_data.duplicated().sum()

0

### Вывод

Поскольку нас нет id клиента, будем считать дубликатами полное совпадение строк. Отбираем дубликаты по всему датафрейму. Такой отбор дает 71 строку. Удаляем дубликаты, в аргументах метода drop_duplicates указываем keep='first', чтобы у нас оставалась первая уникальная строка.

### Лемматизация

In [8]:
from pymystem3 import Mystem
m = Mystem()
purposes_unique = clients_data['purpose'].unique()
for element in purposes_unique:
    try:
        print(m.lemmatize(element))
    except:
        print("Check data in element: purpose= ", element)

#clients_data.groupby('purpose')['debt'].count()
def purposes_new(initial_purpose):
    words=[
        ['жиль', 'недвижимость'],
        ['недвижимост', 'недвижимость'],
        ['автомобил', 'автомобиль'],
        ['образован', 'образование'],
        ['свадьб', 'свадьба']
    ]
    for word in words:
        if word[0] in initial_purpose:
            return word[1]
        
clients_data['purpose_updated'] = clients_data['purpose'].apply(purposes_new)
clients_data.head(10)

['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['жилье', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['автомобиль', '\n']
['заниматься', ' ', 'образование'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_updated
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.93,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.83,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,покупка жилья для семьи,недвижимость


### Вывод

Для того, чтобы понять, на какие цели берут кредиты, используем лемматизацию по столбцу purpose. Получаем список, который можно свести к нескольким основным категориям:

- жилье (объединяем с недвижимостью)
- автомобиль
- образование
- свадьба

Заменяем цели кредитов на выявленные нами категории.

### Категоризация данных

In [9]:
clients_data['years_employed'] = clients_data['days_employed'] / 365
print(clients_data['years_employed'].max())
print(clients_data['years_employed'].min())

#добавляем категории для стажа в годах
def employment_term_category(years):
    if years <= 10:
        return "0-10"
    elif years <= 20:
        return "11-20"
    elif years <= 30:
        return "21-30"
    elif years <=40:
        return "31-40"
    else:
        return "40+"
    
clients_data['employment_category'] = clients_data['years_employed'].apply(employment_term_category)

#добавляем категории возраста
#print(clients_data['dob_years'].min())
#print(clients_data['dob_years'].max())

def age_categorizing(years):
    if years <= 25:
        return "18-25"
    elif years <= 35:
        return "26-35"
    elif years <= 45:
        return "36-45"
    elif years <= 55:
        return "46-55"
    else:
        return "56+"
    
clients_data['age_category'] = clients_data['dob_years'].apply(age_categorizing)

#разбиение доходов по квантилям
income_split = pd.qcut(clients_data['total_income'], 6)
income_split

#добалвяем категории по доходу

def income_categorizing(income):
    if income < 90000:
        return "<90k"
    elif income <= 120000:
        return "90k-120k"
    elif income <= 140000:
        return "100k-140k"
    elif income <= 170000:
        return "140k-170k"
    elif income <= 230000:
        return "170k-230k"
    else:
        return "230k+"
    
clients_data['income_type'] = clients_data['total_income'].apply(income_categorizing)
clients_data.head()

45.85753424657534
0.06575342465753424


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_updated,years_employed,employment_category,age_category
0,1,8437,42,высшее,0,женат / замужем,0,F,230k+,0,253875.64,покупка жилья,недвижимость,23.115068,21-30,36-45
1,1,4024,36,среднее,1,женат / замужем,0,F,90k-120k,0,112080.01,приобретение автомобиля,автомобиль,11.024658,11-20,36-45
2,0,5623,33,среднее,1,женат / замужем,0,M,140k-170k,0,145885.95,покупка жилья,недвижимость,15.405479,11-20,26-35
3,3,4124,32,среднее,1,женат / замужем,0,M,230k+,0,267628.55,дополнительное образование,образование,11.298630,11-20,26-35
4,0,14177,53,среднее,1,гражданский брак,1,F,140k-170k,0,158616.08,сыграть свадьбу,свадьба,38.841096,31-40,46-55


Вывод

Для анализа необходимо разбить заемщиков по категориям. По количеству детей, образованию, семейному статусу и полу можно разбивать с существующими данными.

Для разбивки по возрасту, доходу и стажу необходимо добавлять группы в виде интервалов значений. При выборе интервала отталкиваемся от максимального и минимального значения в таблцие. Для возраста и стажа берем интервал 10 лет, это даст достаточно подробную разбивку и в то же время сводная таблица не будет перегружена. В дальнейшем при необходимости более глубокого анализа эту разбивку можно будет скорректировать.

Для категоризации по доходам необходимо понять, какие интервалы будут оптимальными. Для этого используем функцию qcut. Запустим её с количеством квантилей=6. Видим, что функция возвращает следующие диапазоны:
Categories (6, interval[float64]): [(20667.259, 93170.483] < (93170.483, 120486.357] < (120486.357, 142563.42] < (142563.42, 172396.0] < (172396.0, 229600.233] < (229600.233, 2265604.03]]

таким образом, можно взять следующие интервалы дохода:

- менее 90 тысяч
- 90-120 тысяч
- 120-140 тысяч
- 140-170 тысяч
- 170-230 тысяч
- более 230 тысяч

Пишем функцию с этими интервалами и применяем её к датафрейму.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
clients_data_grouped = clients_data.groupby('children').agg({'debt':['sum', 'count']})
clients_data_grouped['debt_share'] = clients_data_grouped['debt']['sum'] / clients_data_grouped['debt']['count']
clients_data_grouped['debt_share'] = clients_data_grouped['debt_share'].astype('float').map('{:.1%}'.format)
clients_data_grouped.head()

debt        debt_share
           sum  count           
children                        
0         1020  13255       7.7%
1          436   4739       9.2%
2          193   2029       9.5%
3           26    324       8.0%
4            4     40      10.0%

### Вывод

В абсолютных цифрах больше всего просрочек у тех, у кого нет детей. Однако если считать в процентах, лидируют люди с 4 детьми, за ними идут те, у кого 1 или 2 ребенка. Отсюда можно сделать вывод, что люди с детьми имеют более низкую платежную дисциплину.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
clients_data_grouped = clients_data.groupby('family_status').agg({'debt':['sum', 'count']})
clients_data_grouped['debt_share'] = clients_data_grouped['debt']['sum'] / clients_data_grouped['debt']['count']
clients_data_grouped['debt_share'] = clients_data_grouped['debt_share'].astype('float').map('{:.1%}'.format)
clients_data_grouped.head()

debt        debt_share
                       sum  count           
family_status                               
Не женат / не замужем  271   2713      10.0%
в разводе               83   1137       7.3%
вдовец / вдова          56    879       6.4%
гражданский брак       374   3966       9.4%
женат / замужем        903  11773       7.7%

### Вывод

Здесь доля просрочек коррелирует с абсолютными цифрами: неженатый человек либо живущий в гражданском браке чаще просрочивает платежи.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
clients_data_grouped = pd.pivot_table(clients_data, index='income_type', values='debt', aggfunc='sum')
clients_data_grouped['debt_share'] = clients_data_grouped['debt'] / clients_data['debt'].sum()
clients_data_grouped['debt_share'] = clients_data_grouped['debt_share'].astype('float').map('{:.1%}'.format)
clients_data_grouped.head()

,debt,debt_share
income_type,,
100k-140k,188,11.1%
140k-170k,374,22.2%
170k-230k,322,19.1%
230k+,239,14.2%
90k-120k,309,18.3%


### Вывод

На интервал от 140 до 170 тысяч приходится пик просрочек в % от общего количества. Наилучшая платёжная дисциплина у людей с доходом от 100 до 140 тысяч.

- Как разные цели кредита влияют на его возврат в срок?

In [13]:
clients_data_grouped = clients_data.groupby('purpose_updated').agg({'debt':['sum', 'count']})
clients_data_grouped['debt_share'] = clients_data_grouped['debt']['sum'] / clients_data_grouped['debt']['count']
clients_data_grouped['debt_share'] = clients_data_grouped['debt_share'].astype('float').map('{:.1%}'.format)
clients_data_grouped.head()

debt        debt_share
                 sum  count           
purpose_updated                       
автомобиль       390   4128       9.4%
недвижимость     759  10300       7.4%
образование      361   3828       9.4%
свадьба          177   2212       8.0%

### Вывод

Доля невозвратов кредитов на автомобиль и на образование одинаковая. При этом в абсолютных цифрах просроченных займов на недвижимость в 2,5 раза больше.

### Шаг 4. Общий вывод

Согласно проведенному анализу, портрет клиента, который с наименьшей вероятностью вернет займ в срок:

- имеет 1 или несколько детей;
- не женат официально;
- зарабатывает от 150 до 200 тысяч рублей в месяц;
- берет кредит на автомобиль или образование.

В то же время клиент с наилучшей платежной дисциплиной:

- не имеет детей;
- вдовец;
- зарабатывает менее 50 тысяч рублей в месяц;
- цель займа - недвижимость.